# Détermination de la température à l'aide de cristaux liquides : calibration

Exercice créé par Julien Salort du Laboratoire de Physique de l'ENS de Lyon

Le but est de calibrer le lien entre température et couleurs de certains cristaux liquides. Pour cela, on a thermostaté une cuve et réalisé une série d'images pour chaque température. 

Les images sont effectuées avec une caméra  une caméra couleur Ximea XIB64 qui comporte devant son capteur un filtre avec une mosaïque de Bayer GRGB (voir la description dans la documentation d'OpenCV). Le capteur de la caméra a une dynamique de 12 bits, ce qui signifie que les valeurs pour chaque pixel sont comprises entre 0 et $2^{12}-1$. 

Les températures correspondant à chaque expérience (numérotées de 1 à 699) sont stockées dans le fichier `/home/jsalort/rgb/Result_temp_calibration.txt`.

1. Lire le fichier et afficher la température correspondant à chaque expérience. *On rappelle l'utilité de la commande `more` pour avoir une idée de la structure des données.*
## Analyse d'une image ponctuelle
On n'ouvrira pour l'instant que l'image `/home/jsalort/rgb/Run_300/img-cam0-00001.png` qui correspond à l'image numéro 1 pour l'expérience de numéro 300. L'image est initalement du 16 bit monocanal. 


2. Ouvrir l'image avec la librairie `cv2` et la fonction `cv2.imread` [documentation](https://docs.opencv.org/4.x/d4/da8/group__imgcodecs.html#ga288b8b3da0892bd651fce07b3bbd3a56) en utilisant l'option `,cv2.IMREAD_UNCHANGED` ( [documentation](https://docs.opencv.org/4.x/d8/d6a/group__imgcodecs__flags.html#ga61d9b0126a3e57d9277ac48327799c80) )
3. Regarder le shape et le dtype de l'objet `img_raw` obtenu pour constater que l'on a bien une image de 5120 pixels par 5120 pixels monocanal en 16 bits.

Cette image n'est pas lisible par les programmes courants : il faut la convertir en image RGB.

4.  Pour cela, utiliser la fonction `cv2.cvtColor` et utiliser l'option `cv2.COLOR_BayerGR2RGB`. Regarder ensuite le shape de l'objet `img_rgb` pour voir que maintenant, il y a bien les composantes rouges, vertes et bleues. De plus, constater que le dtype est `uint16`

L'image n'est toujours pas visible : il faut la convertir en image sur seulement 8 bits (uint8) au lieu de 16 (uint16). Il faut donc convertir les niveaux compris entre 0 et 4095 en niveaux compris entre 0 et 255. 

5. Convertir `img_rgb` en image 8 bits en se ramenant à des entiers dans l'intervalle 0/255 au lieu de 0/4095. Puis l'afficher avec la fonction `imshow` de matplotlib.

![exo10-img-8bit.png](exo10-img-8bit.png)

## Corrections des composantes et de la luminosité

* Ni la source ni le récepteurs ne sont idéaux. Il faut donc corriger les valeurs pour prendre en compte ces effets : la différence de réponse du capteur ainsi que la possibilité d'avoir un noir pour une valeur supérieur à zéro.
* De plus, la luminosité de la sonde de température « écrase » la luminosité des particules que nous cherchons à étudier. Pour cela, il faut donc écréter les valeurs. Pour cela, on applique des coefficients multiplicateurs sur les composantes rouge (r), verte (g) et bleue (b), et on retranche aussi la valeur du point noir, i.e.
$$v_{R,G,B} = k_{R,G,B}v_{R,G,B}^0 - b_{R,G,B}$$

où :
* $v_{R,G,B}$ est la valeur de la composante R,G ou B de l'image après correction
* $v^0_{R,G,B}$ est la valeur de la composante R,G ou B de l'image après correction
* $k_{R,G,B}$ est le facteur pour prendre en compte les défauts de la caméra et de la source
* $b_{R,G,B}$ est la valeur correspondant au noir pour chacun des canaux

On prendra pour le cas présent les valeurs suivantes:
```
k = np.array([1.75, 1.0, 2.75])
b = np.array([108.61197591026168, 80.8944685895217, 167.05524770245015])
```
pour lesquelles, par exemple : $k_R$ vaut 1,75 et $b_B$ vaut 167,05524770245015.

De plus, si jamais une des valeurs des trois composantes est négative, alors toutes les composantes R, G, B du pixel sont passées à zéro 

6. Faire une fonctions qui part du nom du fichier et retourne une image RGB avec les composantes corrigées, *attention, il faut forcer le type `np.uint16` après le calcul mais les valeurs intermédiaires seront des nombres flottants.*

7. Faire une fonction qui permet d'écréter toutes les valeurs au dessus d'une valeur seuil (on prendra par défaut `seuil=500` puis qui convertit l'image en 8 bits (valeurs entre 0 et 255). *Attention, intermédiairement, il faut faire un calcul en nombre flottant puis à la fin uniquement repasser en type `np.uint8`*

Après les deux corrections, l'image obtenue est la suivante :
![exo10-img-8bit-thresh.png](exo10-img-8bit-thresh.png)

On veut analyser une portion de l'image où la densité de particules est homogène et sans la sonde de température. Pour cela,  on se limite à la zone où x est compris entre 2000 et 4000 et y est compris entre 1000 et 3000.

8. Modifier la fonction créée à la question 6 pour réduire aussi tôt que possible la dimension de l'image et ainsi réduire le coût des calculs.

Les valeurs des composantes RGB obtenues pour le pixel en 0,0 de l'image réduite ainsi obtenue  sont :

`[ 83 120  86]` après correction des composantes RGB

`[18 19 35]` après correction sur 8 bits et écrètage

![exo10-coords-cv.svg](exo10-coords-cv.svg)


## Évolution de la couleur avec la température

La couleur des particules change avec la température. On se propose donc d'étudier le changement de couleur en fonction de la température. Pour chaque expérience, les images sont disponibles dans le fichier `/home/jsalort/rgb/Run_{i}/img-cam0-{j:05d}.png` où `i` est le numéro de l'expérience et `j` correspond au numéro de l'image. En pratique, seules les images `j=1..4` pour les expériences numéros :
* 300 à 309, 
* 330 à 339, 
* 360 à 369, 
* 390 à 399, 
* 420 à 429, 
* 450 à 459, 
* 480 à 489,
* 510 à 519, 
* 540 à 549, 
* 570 à 579

sont copiés sur le home de Julien Salort au CBP.

9. Créer une liste contenant la liste de toutes les expériences. Pour cela, commencer par créer une liste avec tous les indices par pas de 30 (300,330,360, etc) puis ajouter les numéros 0 à 9 pour recréer les numéros. *Challenge : il est possible de le faire sans utiliser de boucle `for` mais avec les fonctions `expand_dims`, et la méthode `flatten()` ou la fonction `reshape`.*

10. Afficher l'image retravaillée pour chaque expérience avec un numéro multiple de 30 (300, 330, etc)

![exo10-img-8bit-thresh-lot.png](exo10-img-8bit-thresh-lot2.png)





En pratique, il est plus facile de travailler en coordonnées HSV (Hue Saturation Value) que RGB pour voir le changement de teinte. Les images étant en 16 bits, la fonction `cv2.cvtColor` de la bibliothèque opencv n'est pas capable de faire la conversion, on va donc utiliser la fonction équivalente : `rgb2hsv` de la bibliothèque sckit-image ( [documentation](https://scikit-image.org/docs/stable/api/skimage.color.html#skimage.color.rgb2hsv) [exemple](https://scikit-image.org/docs/stable/auto_examples/color_exposure/plot_rgb_to_hsv.html) ) :

```python
from skimage.color import rgb2hsv
```

On propose de convertir la couleur RGB dans la base HSV. On est intéressé en particulier par la Hue (teinte). Pour cela, on utilisera la fonction rgb2hsv du module Scikit-image,

11. Convertir les images précédentes en hsv avec la fonction `rgb2hsv` et extraire la composante `h` qui est la première composante de la dernière dimension de l'image ainsi générée. L'histogramme de la teinte des particules sur les différentes images est le suivant (on a pris comme option pour le tracé `,bins=np.linspace(0,1,51)` et encore une fois, il faut aplatir l'image avec la méthode `flatten()` ) :

![exo10-img-hue-lot.png](exo10-img-hue-lot.png)

En pratique, on ne veut prendre en compte que les particules ayant une luminosité suffisante. Elles sont définies commes les particules pour lesquelles la luminosité, prise comme moyenne des composantes R, G et B (en 16 bits), est supérieure à un seuil qui sera pris égal à 300 par défaut.

12. Faire une fonction qui calcule la luminosité avec la fonction `np.mean` (ou `np.average`), qui retourne la luminosite ainsi calculée et qui fournit également le nombre de pixels pour lesquels la luminosité est supérieure au seuil défini. *On pourra utiliser la fonction `np.count_nonzero()` ainsi que du masking*
 Résultat pour un seuil fixé à 300 :
 ```
experiment number : 300 count : 29061
experiment number : 330 count : 28108
experiment number : 360 count : 32716
experiment number : 390 count : 39663
experiment number : 420 count : 41374
experiment number : 450 count : 32165
experiment number : 480 count : 24556
experiment number : 510 count : 20606
experiment number : 540 count : 5181
experiment number : 570 count : 649
 ```
13. Faire une fonction qui calcule la valeur moyenne de la teinte pour les pixels ayant une luminositié supérieure à un seuil défini. 
 Résultat pour un seuil fixé à 300 :
```
experiment number : 300 teinte : 0.5463026280278536
experiment number : 330 teinte : 0.5363894995487174
experiment number : 360 teinte : 0.5249128372747327
experiment number : 390 teinte : 0.517629225790177
experiment number : 420 teinte : 0.4865267723295474
experiment number : 450 teinte : 0.4322647949176736
experiment number : 480 teinte : 0.3482885466779284
experiment number : 510 teinte : 0.18191676376550509
experiment number : 540 teinte : 0.10296949443392385
experiment number : 570 teinte : 0.28199514211808296
```

14. Tracer la teinte en fonction du numéro d'expérience puis de la température et en déduire la gamme de température où les particules peuvent servir d'indicateur de température.

